# RT-DETR Pretraining with SHIFT-Discrete Dataset

## Imports

In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "expandable_segments:True"

In [2]:
from os import path

import torch
from torch import nn, optim

from ttadapters.datasets import BaseDataset, DatasetHolder, DataLoaderHolder
from ttadapters.datasets import SHIFTDiscreteDatasetForObjectDetection
from transformers import Trainer, TrainingArguments, DefaultDataCollator, EarlyStoppingCallback

import numpy as np
import pandas as pd

from tqdm.auto import tqdm
import matplotlib.pyplot as plt

### Check GPU Availability

In [6]:
!nvidia-smi

Thu Jun 26 11:12:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    Off |   00000000:01:00.0 Off |                  N/A |
|  0%   39C    P8             15W /  285W |     931MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 0
ADDITIONAL_GPU = 0

if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        torch.cuda.set_device(DEVICE_NUM)
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

INFO: Using device - cuda:0


## Define Dataset

In [8]:
DATA_ROOT = path.join(".", "data")

dataset = DatasetHolder(
    train=SHIFTDiscreteDatasetForObjectDetection(root=DATA_ROOT, train=True),
    valid=SHIFTDiscreteDatasetForObjectDetection(root=DATA_ROOT, valid=True)
)

[06/26/2025 11:12:39] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/train. Backend: <shift_dev.utils.backend.ZipBackend object at 0x75b4c2008cd0>
[06/26/2025 11:12:39] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT/discrete/images/train/front/det_2d.json' ...


INFO: Downloading 'SHIFT' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[06/26/2025 11:12:40] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT/discrete/images/train/front/det_2d.json' Done.
[06/26/2025 11:12:42] SHIFT DevKit - INFO - Loading annotation takes 3.35 seconds.


Batch 0:

Item                 Shape                               Min        Max       
--------------------------------------------------------------------------------
original_hw          [tensor([800]), tensor([1280])]
input_hw             [tensor([800]), tensor([1280])]
frame_ids            torch.Size([1])                           0.00       0.00
name                 ['00000000_img_front.jpg']
videoName            ['0016-1b62']
intrinsics           torch.Size([1, 3, 3])                     0.00     640.00
extrinsics           torch.Size([1, 4, 4])                    -7.53     219.91
boxes2d              torch.Size([1, 26, 4])                    5.00     974.00
boxes2d_classes      torch.Size([1, 26])                       0.00       3.00
boxes2d_track_ids    torch.Size([1, 26])                       0.00      25.00
images               torch.Size([1, 1, 3, 800, 1280])          0.00     255.00



[06/26/2025 11:12:43] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x75b4c2008cd0>
[06/26/2025 11:12:43] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT/discrete/images/val/front/det_2d.json' ...
[06/26/2025 11:12:43] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT/discrete/images/val/front/det_2d.json' Done.


Video name: 0016-1b62
Sample indices within a video: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
INFO: Downloading 'SHIFT' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[06/26/2025 11:12:43] SHIFT DevKit - INFO - Loading annotation takes 0.24 seconds.


Batch 0:

Item                 Shape                               Min        Max       
--------------------------------------------------------------------------------
original_hw          [tensor([800]), tensor([1280])]
input_hw             [tensor([800]), tensor([1280])]
frame_ids            torch.Size([1])                           0.00       0.00
name                 ['00000000_img_front.jpg']
videoName            ['0116-4859']
intrinsics           torch.Size([1, 3, 3])                     0.00     640.00
extrinsics           torch.Size([1, 4, 4])                    -0.90     138.34
boxes2d              torch.Size([1, 6, 4])                   246.00     859.00
boxes2d_classes      torch.Size([1, 6])                        1.00       5.00
boxes2d_track_ids    torch.Size([1, 6])                        0.00       5.00
images               torch.Size([1, 1, 3, 800, 1280])          0.00     255.00

Video name: 0116-4859
Sample indices within a video: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,

In [9]:
dataset.train[1]['front'].keys()

dict_keys(['original_hw', 'input_hw', 'frame_ids', 'name', 'videoName', 'intrinsics', 'extrinsics', 'boxes2d', 'boxes2d_classes', 'boxes2d_track_ids', 'images'])

In [10]:
dataset.train[999]

{'front': {'original_hw': (800, 1280),
  'input_hw': (800, 1280),
  'frame_ids': 490,
  'name': '00000490_img_front.jpg',
  'videoName': '0c9d-eefc',
  'intrinsics': tensor([[640.,   0., 640.],
          [  0., 640., 400.],
          [  0.,   0.,   1.]]),
  'extrinsics': tensor([[-5.7429e-01,  7.7804e-01, -2.5465e-01,  1.6100e+02],
          [-7.0979e-01, -6.2821e-01, -3.1867e-01, -2.0023e+01],
          [-4.0791e-01, -2.2626e-03,  9.1302e-01,  1.5929e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
  'boxes2d': tensor([[ 457.,  405.,  525.,  467.],
          [ 599.,  391.,  612.,  403.],
          [ 599.,  398.,  677.,  459.],
          [ 835.,  391., 1280.,  605.],
          [ 655.,  396.,  668.,  402.],
          [ 392.,  394.,  404.,  401.],
          [ 665.,  396.,  676.,  402.],
          [ 842.,  390.,  848.,  397.],
          [1207.,  380., 1217.,  399.]]),
  'boxes2d_classes': tensor([1, 2, 1, 1, 1, 2, 1, 0, 0]),
  'boxes2d_track_ids': tensor([ 4,  1,  0

In [11]:
dataset.train[1000]['front']['images'].shape  # should be (batch_size, num_channels, height, width)

torch.Size([1, 3, 800, 1280])

## DataLoader

In [12]:
# Set Batch Size
BATCH_SIZE = 32, 20, 32

# Dataset Configs
CLASSES = dataset.train.classes
NUM_CLASSES = len(CLASSES)

print(f"INFO: Set batch size - Train: {BATCH_SIZE[0]}, Valid: {BATCH_SIZE[1]}, Test: {BATCH_SIZE[2]}")
print(f"INFO: Number of classes - {NUM_CLASSES} {CLASSES}")

INFO: Set batch size - Train: 32, Valid: 20, Test: 32
INFO: Number of classes - 6 ['pedestrian', 'car', 'truck', 'bus', 'motorcycle', 'bicycle']


In [13]:
class DatasetAdapterForTransformers(BaseDataset):
    def __init__(self, original_dataset, camera='front'):
        self.dataset = original_dataset
        self.camera = camera

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx][self.camera]
        image = item['images'].squeeze(0)

        # Convert to COCO_Detection Format
        annotations = []
        target = dict(image_id=idx, annotations=annotations)
        for box, cls in zip(item['boxes2d'], item['boxes2d_classes']):
            x1, y1, x2, y2 = box.tolist()
            width, height = x2 - x1, y2 - y1
            annotations.append(dict(
                bbox=[x1, y1, width, height],  # COCO format: [x, y, width, height]
                category_id=cls.item(),
                area=width * height,
                iscrowd=0
            ))

        # Following prepare_coco_detection_annotation's expected format
        # But, RT-DETR ImageProcessor eventually re-converts the bbox to (x1, y1, x2, y2) format
        return dict(image=image, target=target)

In [14]:
def collate_fn(batch, preprocessor=None):
    images = [item['image'] for item in batch]
    if preprocessor is not None:
        target = [item['target'] for item in batch]
        return preprocessor(images=images, annotations=target, return_tensors="pt")
    else:
        # If no preprocessor is provided, just assume images are already in tensor format
        return dict(
            pixel_values=dict(pixel_values=torch.stack(images)),
            labels=[dict(
                class_labels=item['boxes2d_classes'].long(),
                boxes=item["boxes2d"].float()
            ) for item in batch]
        )

## Load Model

In [15]:
from transformers import RTDetrForObjectDetection, RTDetrImageProcessorFast, RTDetrConfig
from transformers.image_utils import AnnotationFormat

In [16]:
reference_model_id = "PekingU/rtdetr_r50vd"

# Load the reference model configuration
reference_config = RTDetrConfig.from_pretrained(reference_model_id)
reference_config.num_labels = NUM_CLASSES

# Load the reference model image processor
reference_preprocessor = RTDetrImageProcessorFast.from_pretrained(reference_model_id)
reference_preprocessor.format = AnnotationFormat.COCO_DETECTION  # COCO Format / Detection BBOX Format

In [17]:
model = RTDetrForObjectDetection(config=reference_config)

if ADDITIONAL_GPU:
    model = nn.DataParallel(model, device_ids=list(range(DEVICE_NUM, DEVICE_NUM+ADDITIONAL_GPU+1)))
model.to(device)

RTDetrForObjectDetection(
  (model): RTDetrModel(
    (backbone): RTDetrConvEncoder(
      (model): RTDetrResNetBackbone(
        (embedder): RTDetrResNetEmbeddings(
          (embedder): Sequential(
            (0): RTDetrResNetConvLayer(
              (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (1): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (2): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
          )
          (pooler): MaxPool2d(

In [18]:
test_d = DatasetAdapterForTransformers(dataset.train)[5]
test_d

{'image': tensor([[[ 64.,  65.,  67.,  ..., 121., 121., 121.],
          [ 62.,  61.,  64.,  ..., 121., 121., 121.],
          [ 58.,  60.,  61.,  ..., 121., 121., 121.],
          ...,
          [105., 105., 105.,  ...,  60.,  64.,  72.],
          [103., 103., 103.,  ...,  67.,  67.,  67.],
          [100., 100., 100.,  ...,  76.,  72.,  66.]],
 
         [[ 27.,  26.,  26.,  ..., 139., 139., 139.],
          [ 25.,  24.,  22.,  ..., 139., 139., 139.],
          [ 22.,  23.,  22.,  ..., 139., 139., 139.],
          ...,
          [106., 106., 106.,  ...,  59.,  63.,  71.],
          [104., 104., 104.,  ...,  66.,  66.,  66.],
          [101., 101., 101.,  ...,  75.,  71.,  65.]],
 
         [[  9.,   9.,   8.,  ..., 153., 153., 153.],
          [  7.,   6.,   6.,  ..., 153., 153., 153.],
          [  6.,   7.,   7.,  ..., 153., 153., 153.],
          ...,
          [111., 111., 111.,  ...,  65.,  69.,  77.],
          [109., 109., 109.,  ...,  72.,  72.,  72.],
          [106., 106.,

In [19]:
reference_preprocessor(images=test_d['image'], annotations=test_d['target'])

{'pixel_values': tensor([[[[0.2514, 0.2654, 0.2874,  ..., 0.4745, 0.4745, 0.4745],
          [0.2384, 0.2522, 0.2799,  ..., 0.4745, 0.4745, 0.4745],
          [0.2345, 0.2456, 0.2704,  ..., 0.4768, 0.4768, 0.4768],
          ...,
          [0.4108, 0.4118, 0.4114,  ..., 0.2585, 0.2519, 0.2749],
          [0.4072, 0.4083, 0.4117,  ..., 0.2805, 0.2562, 0.2625],
          [0.3951, 0.3963, 0.4037,  ..., 0.3235, 0.2901, 0.2716]],

         [[0.1014, 0.0963, 0.1000,  ..., 0.5451, 0.5451, 0.5451],
          [0.0919, 0.0887, 0.0952,  ..., 0.5451, 0.5451, 0.5451],
          [0.0912, 0.0901, 0.0929,  ..., 0.5474, 0.5474, 0.5474],
          ...,
          [0.4147, 0.4157, 0.4154,  ..., 0.2545, 0.2480, 0.2710],
          [0.4111, 0.4123, 0.4156,  ..., 0.2766, 0.2522, 0.2586],
          [0.3990, 0.4002, 0.4076,  ..., 0.3196, 0.2862, 0.2676]],

         [[0.0324, 0.0304, 0.0396,  ..., 0.6000, 0.6000, 0.6000],
          [0.0254, 0.0253, 0.0359,  ..., 0.6000, 0.6000, 0.6000],
          [0.0317, 0.0325

In [20]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval


def compute_coco_metrics(eval_pred):
    predictions, labels = eval_pred

    coco_gt = COCO()  # ground truth
    coco_dt = coco_gt.loadRes(predictions)  # predictions

    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    return {
        'mAP@0.95': coco_eval.stats[0],
        'mAP@0.5': coco_eval.stats[1],
        'mAP@0.75': coco_eval.stats[2],
        'mAP_small': coco_eval.stats[3],
        'mAP_medium': coco_eval.stats[4],
        'mAP_large': coco_eval.stats[5],
        'AR@1': coco_eval.stats[6],
        'AR@10': coco_eval.stats[7],
        'AR@100': coco_eval.stats[8],
        'AR_small': coco_eval.stats[9],
        'AR_medium': coco_eval.stats[10],
        'AR_large': coco_eval.stats[11],
    }

In [21]:
archived = []

In [22]:
from torchmetrics.detection import MeanAveragePrecision

def compute_coco_metrics(eval_pred):
    predictions, labels = eval_pred
    archived.append(predictions)
    archived.append(labels)

    # torchmetrics MeanAveragePrecision 객체 생성
    # COCO와 동일한 설정
    metric = MeanAveragePrecision(
        box_format='cxcywh',  # RT-DETR는 center format 사용
        iou_type='bbox',
        iou_thresholds=None,  # COCO 기본값 사용 (0.5:0.05:0.95)
        rec_thresholds=None,  # COCO 기본값 사용
        max_detection_thresholds=[1, 10, 100],  # COCO 기본값
        class_metrics=False,  # 전체 평균만 계산
        sync_on_compute=True
    )

    # predictions와 labels를 torchmetrics 형식으로 변환
    preds = []
    targets = []

    for i, (pred, label) in enumerate(zip(predictions, labels)):
        # predictions 처리 (RT-DETR 출력에 맞게 조정 필요)
        # RT-DETR 출력 형식: logits, pred_boxes
        if isinstance(pred, dict):
            # 모델 출력에서 boxes, scores, labels 추출
            pred_dict = {
                'boxes': pred.get('pred_boxes', torch.empty(0, 4)),
                'scores': pred.get('scores', torch.empty(0)),
                'labels': pred.get('labels', torch.empty(0, dtype=torch.long))
            }
        else:
            # 임시 빈 예측 (실제로는 RT-DETR 출력 파싱 필요)
            pred_dict = {
                'boxes': torch.empty(0, 4),
                'scores': torch.empty(0),
                'labels': torch.empty(0, dtype=torch.long)
            }

        # labels 처리 (이미 올바른 형식으로 되어 있음)
        if isinstance(label, dict):
            target_dict = {
                'boxes': label.get('boxes', torch.empty(0, 4)),
                'labels': label.get('class_labels', torch.empty(0, dtype=torch.long))
            }
        else:
            target_dict = {
                'boxes': torch.empty(0, 4),
                'labels': torch.empty(0, dtype=torch.long)
            }

    preds.append(pred_dict)
    targets.append(target_dict)

    metric.update(preds, targets)
    result = metric.compute()

    return {
        'mAP@0.95': result['map'].item(),           # mAP @ IoU=0.50:0.95
        'mAP@0.5': result['map_50'].item(),         # mAP @ IoU=0.50
        'mAP@0.75': result['map_75'].item(),        # mAP @ IoU=0.75
        'mAP_small': result['map_small'].item(),    # mAP for small objects
        'mAP_medium': result['map_medium'].item(),  # mAP for medium objects
        'mAP_large': result['map_large'].item(),    # mAP for large objects
        'AR@1': result['mar_1'].item(),             # AR given 1 detection per image
        'AR@10': result['mar_10'].item(),           # AR given 10 detections per image
        'AR@100': result['mar_100'].item(),         # AR given 100 detections per image
        'AR_small': result['mar_small'].item(),     # AR for small objects
        'AR_medium': result['mar_medium'].item(),   # AR for medium objects
        'AR_large': result['mar_large'].item(),     # AR for large objects
    }

In [23]:
# Set Epoch Count & Learning Rate
EPOCHS = 15
LEARNING_RATE = 2e-5

training_args = TrainingArguments(
    learning_rate=LEARNING_RATE,
    warmup_ratio=0.1,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE[0],
    per_device_eval_batch_size=BATCH_SIZE[1],
    eval_accumulation_steps=BATCH_SIZE[1],
    remove_unused_columns=False,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    eval_strategy="steps",
    eval_steps=100,
    save_steps=100,
    save_total_limit=10,
    save_strategy="steps",
    load_best_model_at_end=True,
    #metric_for_best_model="mAP@0.95",
    #greater_is_better=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    #report_to="wandb",
    output_dir="./results",
    logging_dir="./logs",
    fp16=True,
)

In [24]:
from functools import partial

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=DatasetAdapterForTransformers(dataset.train),
    eval_dataset=DatasetAdapterForTransformers(dataset.valid),
    data_collator=partial(collate_fn, preprocessor=reference_preprocessor),
    compute_metrics=compute_coco_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

evaluator = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./results",
        per_device_eval_batch_size=BATCH_SIZE[2],
        dataloader_drop_last=False,
        do_train=False,
        do_eval=True,
    ),
    train_dataset=DatasetAdapterForTransformers(dataset.train),
    eval_dataset=DatasetAdapterForTransformers(dataset.valid),
    data_collator=partial(collate_fn, preprocessor=reference_preprocessor),
    compute_metrics=compute_coco_metrics
)

## Train

In [22]:
trainer.train(resume_from_checkpoint=True)
#trainer.train()

There were missing keys in the checkpoint model loaded: ['class_embed.0.weight', 'class_embed.0.bias', 'class_embed.1.weight', 'class_embed.1.bias', 'class_embed.2.weight', 'class_embed.2.bias', 'class_embed.3.weight', 'class_embed.3.bias', 'class_embed.4.weight', 'class_embed.4.bias', 'class_embed.5.weight', 'class_embed.5.bias', 'bbox_embed.0.layers.0.weight', 'bbox_embed.0.layers.0.bias', 'bbox_embed.0.layers.1.weight', 'bbox_embed.0.layers.1.bias', 'bbox_embed.0.layers.2.weight', 'bbox_embed.0.layers.2.bias', 'bbox_embed.1.layers.0.weight', 'bbox_embed.1.layers.0.bias', 'bbox_embed.1.layers.1.weight', 'bbox_embed.1.layers.1.bias', 'bbox_embed.1.layers.2.weight', 'bbox_embed.1.layers.2.bias', 'bbox_embed.2.layers.0.weight', 'bbox_embed.2.layers.0.bias', 'bbox_embed.2.layers.1.weight', 'bbox_embed.2.layers.1.bias', 'bbox_embed.2.layers.2.weight', 'bbox_embed.2.layers.2.bias', 'bbox_embed.3.layers.0.weight', 'bbox_embed.3.layers.0.bias', 'bbox_embed.3.layers.1.weight', 'bbox_embed.3.l

Step,Training Loss,Validation Loss


TrainOutput(global_step=20800, training_loss=0.0, metrics={'train_runtime': 0.0039, 'train_samples_per_second': 133945715.163, 'train_steps_per_second': 4185803.599, 'total_flos': 2.099592177010606e+20, 'train_loss': 0.0, 'epoch': 32.0})

In [ ]:
trainer.evaluate()

In [ ]:
archived[0]

In [ ]:
len(archived[0])

In [ ]:
archived[1]

In [ ]:
len(archived[1])

In [ ]:
archived[1][1]